In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.cluster import KMeans


In [2]:
inputfile = '../data/test/mexico_k_1_layers_5.csv'
profile_file = '../data/profiles.csv'

def get_data():
    profiles_file = pd.read_csv(profile_file)
    profiles_file = profiles_file[['profile_id', 'cwrb_reference_soil_group']]
    data = pd.read_csv(inputfile)
    data = profiles_file.merge(data, how="inner", left_on=[
        'profile_id'], right_on=['profile_id'])

    data = remove_small_classes(data, 15)
    
    
    
    data = remove_outliers(data)


    y = data.cwrb_reference_soil_group.astype(str)
    X = data.drop(['cwrb_reference_soil_group'], axis=1)

    return X, y, data

def remove_small_classes(df, min):
    uniques = df.cwrb_reference_soil_group.unique()
    for u in uniques:
        cnt = df[df.cwrb_reference_soil_group == u].shape[0]
        if cnt < min:
            df = df[df.cwrb_reference_soil_group != u]
            print('Deleting {} with {} occurrences'.format(u, cnt))

    return df


# Add Results from Classifiers

In [3]:

X, y, final_data = get_data()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y)


Deleting Plinthosols with 6 occurrences
Deleting Histosols with 10 occurrences
Removing 0 outliers


In [4]:
clf_RF = RandomForestClassifier(min_samples_split=6,
                                n_estimators=1300, min_samples_leaf=2,
                                oob_score=True, class_weight="balanced", n_jobs=2)

clf_RF.fit(X_train.drop('profile_id', axis=1), y_train)

preds_RF_val = clf_RF.predict(X_test.drop('profile_id', axis=1))
preds_RF_train = clf_RF.predict(X_train.drop('profile_id', axis=1))

print(cohen_kappa_score(preds_RF_val, y_test))


0.47055484024855365


In [5]:
clf_XGB = XGBClassifier(n_estimators=300, min_child_weight=7,
                    gamma=0.2, subsample=0.8, colsample_bytree=0.8, n_jobs=-1)

clf_XGB.fit(X_train.drop('profile_id', axis=1), y_train)

preds_XGB_val = clf_XGB.predict(X_test.drop('profile_id', axis=1))
preds_XGB_train = clf_XGB.predict(X_train.drop('profile_id', axis=1))

print(cohen_kappa_score(preds_XGB_val, y_test))


0.4874200504590799


In [6]:
data_val = X_test.copy()
data_val['pred_RF'] = preds_RF_val
data_val['pred_XGB'] = preds_XGB_val
data_val['pred_type'] = 'validation'

data_train = X_train.copy()
data_train['pred_RF'] = preds_RF_train
data_train['pred_XGB'] = preds_XGB_train
data_train['pred_type'] = 'train'

data_all = pd.concat([data_train, data_val])

final_data = final_data.merge(data_all, how="inner")


# Add Clustering data

In [7]:
# Add to df
for cluster in range(4, 9):
    print(f'Testing k = {cluster}')
    kmeans = KMeans(n_clusters=cluster, init="k-means++")
    kmeans.fit(final_data.drop(['profile_id','cwrb_reference_soil_group',
                                 'pred_RF', 'pred_XGB', 'pred_type', 'longitude','latitude'], axis=1))
    final_data[f'cluster_{cluster}']= kmeans.predict(final_data.drop(['profile_id','cwrb_reference_soil_group',
                                 'pred_RF', 'pred_XGB', 'pred_type', 'longitude','latitude'], axis=1))

Testing k = 4
Testing k = 5
Testing k = 6
Testing k = 7
Testing k = 8


In [8]:
# Add Not normalized lat long

profiles = pd.read_csv(profile_file)
profiles = profiles[['profile_id', 'latitude', 'longitude']]

final_data.rename(index=str, columns={"latitude": "latitude_standard", "longitude": "longitude_standard"}, inplace=True)
final_data = final_data.merge(profiles, how="inner", left_on=[
        'profile_id'], right_on=['profile_id'])


In [9]:
final_data.head(10)

,profile_id,cwrb_reference_soil_group,upper_depth,lower_depth,tceq_value_avg,clay_value_avg,elcosp_value_avg,orgc_value_avg,phaq_value_avg,sand_value_avg,...,pred_RF,pred_XGB,pred_type,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,latitude,longitude
0,205798,Regosols,-0.934569,-0.830317,-0.47501,-1.383935,-0.178067,0.433681,-0.338778,1.969068,...,Arenosols,Regosols,validation,3,1,2,3,3,16.008696,-97.413122
1,205801,Regosols,-0.934569,-1.080485,-0.47501,-1.497614,-0.178067,-0.288996,-1.309646,1.969068,...,Regosols,Regosols,train,3,1,2,3,3,15.988968,-97.308131
2,205803,Cambisols,-0.934569,-1.080485,-0.47501,-0.929220,-0.178067,0.630775,-1.066929,0.719543,...,Cambisols,Cambisols,validation,3,1,2,3,3,15.966852,-96.244683
3,205811,Regosols,-0.934569,-1.191671,-0.47501,-1.383935,-0.178067,0.039494,-0.581495,1.776833,...,Regosols,Regosols,train,3,1,2,3,3,15.907219,-97.042755
4,205820,Cambisols,-0.934569,-0.941503,-0.47501,-1.383935,-0.178067,0.433681,-0.662401,1.872950,...,Cambisols,Cambisols,train,3,1,2,3,3,15.846097,-96.958599
5,205821,Regosols,-0.934569,-0.941503,-0.47501,-0.247148,-0.178067,-0.091902,-0.743306,0.335074,...,Regosols,Regosols,train,0,4,1,5,4,15.846002,-96.722743
6,205826,Regosols,-0.934569,-1.163875,-0.47501,-0.701863,-0.178067,-0.026204,-1.066929,1.007895,...,Regosols,Regosols,train,0,4,1,5,4,15.828704,-96.185250
7,205828,Regosols,-0.934569,-0.885910,-0.47501,-1.383935,-0.178067,-0.223298,-1.471457,0.815660,...,Leptosols,Regosols,validation,0,4,1,5,4,15.814320,-96.718176
8,205829,Regosols,-0.934569,-1.024892,-0.47501,-1.383935,-0.178067,0.630775,-1.228740,0.911777,...,Regosols,Regosols,train,3,1,2,3,3,15.808594,-96.429577
9,205830,Regosols,-0.934569,-0.552351,-0.47501,-1.383935,-0.178067,-0.288996,-0.986023,1.872950,...,Regosols,Regosols,train,3,1,2,3,3,15.802994,-96.883041


In [11]:
final_data.to_csv('mexico_k_1_layers_5_with_results.csv', index=False)


# Visualize RF

In [15]:


estimator = clf_RF.estimators_[5]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = list(X_train.drop('profile_id', axis=1).columns),
                class_names = clf_RF.,
                max_depth=5,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

